In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("what's something funny about woodpeckers")

Joke(setup='Why did the woodpecker get a promotion?', punchline='Because he was always pecking away at his work!', rating=7)

In [26]:
from langchain_core.prompts import ChatPromptTemplate

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

human: Tell me a joke about planes
AI: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

human: Tell me another joke about planes
AI: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

human: Now about caterpillars
AI: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
print(few_shot_structured_llm.invoke("what's something funny about woodpeckers"))
print(prompt.messages[0].format_messages()[0])

setup='Woodpecker' punchline='Woodpecker knocks all day, but who knew it was just trying to get some woodblock party vibes going?' rating=7
content='You are a hilarious comedian. Your specialty is knock-knock jokes. Return a joke which has the setup (the response to "Who\'s there?") and the final punchline (the response to "<setup> who?").\n\nHere are some examples of jokes:\n\nhuman: Tell me a joke about planes\nAI: {"setup": "Why don\'t planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}\n\nhuman: Tell me another joke about planes\nAI: {"setup": "Cargo", "punchline": "Cargo \'vroom vroom\', but planes go \'zoom zoom\'!", "rating": 10}\n\nhuman: Now about caterpillars\nAI: {"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}' additional_kwargs={} response_metadata={}


In [56]:
tools = [Joke]
llm_with_tool = llm.bind_tools(tools)
few_shot_llm_with_tools = prompt | llm_with_tool
few_shot_llm_with_tools.invoke("what's something funny about woodpeckers").to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'messages', 'AIMessage'],
 'kwargs': {'content': '',
  'additional_kwargs': {'tool_calls': [{'id': 'call_Yeg65UF0PmFwVbMCwHex5j78',
     'function': {'arguments': '{"setup":"Woodpecker","punchline":"Woodpecker knocking on trees, just trying to get a little \'bark\' in my life!","rating":7}',
      'name': 'Joke'},
     'type': 'function'}],
   'refusal': None},
  'response_metadata': {'token_usage': {'completion_tokens': 46,
    'prompt_tokens': 278,
    'total_tokens': 324,
    'completion_tokens_details': {'reasoning_tokens': 0}},
   'model_name': 'gpt-4o-mini-2024-07-18',
   'system_fingerprint': 'fp_1bb46167f9',
   'finish_reason': 'tool_calls',
   'logprobs': None},
  'type': 'ai',
  'id': 'run-956cd3ac-50bf-45e6-be32-ea13d6b17a6d-0',
  'tool_calls': [{'name': 'Joke',
    'args': {'setup': 'Woodpecker',
     'punchline': "Woodpecker knocking on trees, just trying to get a little 'bark' in my life!",
     'rating': 7}

In [50]:
await few_shot_llm_with_tools.ainvoke("what's something funny about woodpeckers")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hEZ0vS7qfLALJ8jx6LBPehnB', 'function': {'arguments': '{"setup":"Woodpecker","punchline":"Woodpecker knocked on my door, I told him to \'peck off\'!","rating":7}', 'name': 'Joke'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 278, 'total_tokens': 321, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fb74391e-d00a-42a7-b3fc-4959999971a3-0', tool_calls=[{'name': 'Joke', 'args': {'setup': 'Woodpecker', 'punchline': "Woodpecker knocked on my door, I told him to 'peck off'!", 'rating': 7}, 'id': 'call_hEZ0vS7qfLALJ8jx6LBPehnB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 278, 'output_tokens': 43, 'total_tokens': 321})